In [2]:
pip install pygam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 12.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 12.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.
Note: you may need to

In [130]:
from pygam import LinearGAM, s, f
import pandas as pd
import patsy as pt
import numpy as np
from plotly import tools
import plotly.offline as py
import plotly.graph_objs as go
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing
import matplotlib.pyplot as plt

# Prep the dataset
#   Put this back on one line!!
data = pd.read_csv("https://raw.githubusercontent.com/jacobaschoff/midterm/main/nationwide-drugs-fy20-fy23.csv")

In [79]:
data = pd.DataFrame(data)

In [80]:
data

,FY,Month (abbv),Component,Region,Land Filter,Area of Responsibility,Drug Type,Count of Event,Sum Qty (lbs)
0,2020,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Other Drugs**,2,0.330693
1,2020,APR,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Ketamine,1,0.017417
2,2020,APR,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Khat (Catha Edulis),2,1137.497087
3,2020,APR,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Other Drugs**,13,5.258907
4,2020,APR,Office of Field Operations,Coastal/Interior,Other,CHICAGO FIELD OFFICE,Cocaine,7,125.697881
...,...,...,...,...,...,...,...,...,...
9919,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,TUCSON SECTOR,Other Drugs**,4,0.222700
9920,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Fentanyl,2,4.952400
9921,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Marijuana,11,0.412100
9922,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Methamphetamine,2,84.452600


In [81]:
data['datetime'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(data['FY'], data['Month (abbv)'])])

In [82]:
data = data.sort_values('datetime').reset_index(drop=True)
data = data.set_index(['datetime'])

In [83]:
fentanylData = data[data['Drug Type'] == 'Fentanyl']

In [84]:
fentanylData

,FY,Month (abbv),Component,Region,Land Filter,Area of Responsibility,Drug Type,Count of Event,Sum Qty (lbs)
datetime,,,,,,,,,
2020-01-01,2020,JAN,Office of Field Operations,Northern Border,Other,DETROIT FIELD OFFICE,Fentanyl,1,2.579408
2020-01-01,2020,JAN,Office of Field Operations,Coastal/Interior,Other,CHICAGO FIELD OFFICE,Fentanyl,8,3.905930
2020-01-01,2020,JAN,Office of Field Operations,Coastal/Interior,Other,MIAMI FIELD OFFICE,Fentanyl,3,0.006614
2020-01-01,2020,JAN,U.S. Border Patrol,Southwest Border,Land Only,EL CENTRO SECTOR,Fentanyl,1,0.753000
2020-01-01,2020,JAN,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Fentanyl,2,3.159000
...,...,...,...,...,...,...,...,...,...
2023-12-01,2023,DEC,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,1,1.322800
2023-12-01,2023,DEC,Office of Field Operations,Southwest Border,Land Only,LAREDO FIELD OFFICE,Fentanyl,2,7.407532
2023-12-01,2023,DEC,Office of Field Operations,Southwest Border,Land Only,EL PASO FIELD OFFICE,Fentanyl,5,45.136099


In [151]:
fenD = fentanylData[fentanylData['Land Filter'] == 'Land Only']

In [152]:
fenD1 = fentanylData[fentanylData['Area of Responsibility'] == 'SAN DIEGO SECTOR']

In [153]:
fenD1.columns

Index(['FY', 'Month (abbv)', 'Component', 'Region', 'Land Filter',
       'Area of Responsibility', 'Drug Type', 'Count of Event',
       'Sum Qty (lbs)'],
      dtype='object')

In [158]:
fenD1A = fenD1['Sum Qty (lbs)']
fenD1

,FY,Month (abbv),Component,Region,Land Filter,Area of Responsibility,Drug Type,Count of Event,Sum Qty (lbs)
datetime,,,,,,,,,
2020-01-01,2020,JAN,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,2,39.5726
2020-02-01,2020,FEB,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,2,5.5106
2020-03-01,2020,MAR,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,2,15.2558
2020-04-01,2020,APR,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,1,0.5503
2020-05-01,2020,MAY,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,2,0.5644
2020-06-01,2020,JUN,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,4,26.4050
2020-07-01,2020,JUL,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,4,23.3038
2020-08-01,2020,AUG,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,3,223.8742
2020-09-01,2020,SEP,U.S. Border Patrol,Southwest Border,Land Only,SAN DIEGO SECTOR,Fentanyl,3,0.2099


In [155]:
px.line(fenD1, x=fenD1.index.values, y='Sum Qty (lbs)',
       labels = {
           'x' : 'Date',
           'Sum Qty (lbs)' : 'Weight (lbs)'
       })

In [165]:
model = SimpleExpSmoothing(np.asarray(fenD1['Sum Qty (lbs)']), model='add').fit()
model._index = pd.to_datetime(fenD1.index)

TypeError: SimpleExpSmoothing.__init__() got an unexpected keyword argument 'model'

In [163]:
model

In [25]:
data['datetime'] = pd.to_datetime([f'{y}-{m}-01' for y, m in zip(data['FY'], data['Month (abbv)'])])

In [26]:
data

,FY,Month (abbv),Component,Region,Land Filter,Area of Responsibility,Drug Type,Count of Event,Sum Qty (lbs),datetime
0,2020,APR,Office of Field Operations,Coastal/Interior,Other,ATLANTA FIELD OFFICE,Other Drugs**,2,0.330693,2020-04-01
1,2020,APR,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Ketamine,1,0.017417,2020-04-01
2,2020,APR,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Khat (Catha Edulis),2,1137.497087,2020-04-01
3,2020,APR,Office of Field Operations,Coastal/Interior,Other,BALTIMORE FIELD OFFICE,Other Drugs**,13,5.258907,2020-04-01
4,2020,APR,Office of Field Operations,Coastal/Interior,Other,CHICAGO FIELD OFFICE,Cocaine,7,125.697881,2020-04-01
...,...,...,...,...,...,...,...,...,...,...
9919,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,TUCSON SECTOR,Other Drugs**,4,0.222700,2023-09-01
9920,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Fentanyl,2,4.952400,2023-09-01
9921,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Marijuana,11,0.412100,2023-09-01
9922,2023,SEP,U.S. Border Patrol,Southwest Border,Land Only,YUMA SECTOR,Methamphetamine,2,84.452600,2023-09-01
